In [2]:
import os

In [3]:
%pwd

'C:\\Users\\DIKSHANT PATEL\\Kidney-Disease-Classification\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'C:\\Users\\DIKSHANT PATEL\\Kidney-Disease-Classification'

In [6]:
import tensorflow as tf

In [6]:
optuna_best_model = tf.keras.models.load_model("artifacts/optuna_best_model/best_model.keras")

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class OptunaTestingConfig:
    optuna_best_trained_model: str
    test_data: str
    mlflow_uri: str
    optuna_test_scores: str


In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json


In [9]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_roots])

    def get_optuna_test_config(self) -> OptunaTestingConfig:
        config = self.config.optuna_testing_model
        return OptunaTestingConfig(
            optuna_best_trained_model=config.optuna_best_trained_model,
            test_data=config.test_data,
            mlflow_uri=config.mlflow_uri,
            optuna_test_scores = config.optuna_test_scores
        )

In [16]:
import mlflow.keras
from urllib.parse import urlparse
import dagshub
from pathlib import Path
from cnnClassifier.utils.common import create_directories

In [23]:
class OptunaEvaluation:
    def __init__(self, config: OptunaTestingConfig, test_generator):
        self.config = config
        self.test_generator = test_generator
        logger.info("Initialized OptunaEvaluation with given configuration.")


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        logger.info(f"Loading model from path: {path}")
        return tf.keras.models.load_model(path)


    def evaluate(self):
        logger.info("Starting model evaluation on test dataset...")
        self.model = self.load_model(Path(self.config.optuna_best_trained_model))
        self.score = self.model.evaluate(self.test_generator, verbose=1)
        logger.info(f"Model evaluation completed. Loss: {self.score[0]}, Accuracy: {self.score[1]}")
        self.save_score()

    def save_score(self):
        scores = {"optuna_test_loss": self.score[0], "optuna_test_accuracy": self.score[1]}
        score_path = Path(self.config.optuna_test_scores)
        create_directories([score_path.parent])
        save_json(score_path, data=scores)
        logger.info(f"Saved evaluation scores at: {score_path}")

    def log_into_mlflow(self):
        logger.info("Logging evaluation results into MLflow...")
        dagshub.init(repo_owner='DikshantPatel2210', repo_name='KidneyDiseaseClassification-DLProject', mlflow=True)
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        mlflow.set_registry_uri(self.config.mlflow_uri)
        mlflow.set_experiment("Optuna-BestModel-TestEval-v1")
        

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run(run_name="Optuna-BestModel-TestEval"):
            mlflow.log_param("model_type", "OptunaBest")
            mlflow.log_metrics({"optuna_test_loss": self.score[0], "optuna_test_accuracy": self.score[1]})
            logger.info("Logged parameters and metrics to MLflow.")

            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="OptunaBestModel")
                logger.info("Model registered to MLflow Registry.")
            else:
                mlflow.keras.log_model(self.model, "model")
                logger.info("Model logged to MLflow without registration.")

In [24]:
from cnnClassifier.config.configuration import ConfigurationManager
from cnnClassifier.components.data_loader import DataLoader
#from cnnClassifier.entity.config_entity import OptunaTestingConfig
#from cnnClassifier.components.optuna_evaluation import OptunaEvaluation  # adjust import path accordingly
from cnnClassifier import logger

In [25]:
try:
    config_manager = ConfigurationManager()
    
    # Load test generator
    data_loader_config = config_manager.get_data_loader_config()
    data_loader = DataLoader(data_loader_config)
    _, _, test_generator, _, _ = data_loader.get_generators()
    
    # Get optuna test configuration
    optuna_test_config = config_manager.get_optuna_test_config()

    # Initialize evaluator and run evaluation + logging
    evaluator = OptunaEvaluation(optuna_test_config, test_generator)
    evaluator.evaluate()
    evaluator.log_into_mlflow()

except Exception as e:
    logger.error(f"Exception occurred during Optuna model evaluation: {str(e)}")
    raise e


[2025-05-15 12:53:30,445: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-15 12:53:30,457: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-15 12:53:30,459: INFO: common: created directory at: artifacts]
[2025-05-15 12:53:30,461: INFO: data_loader: Loading dataframe from file: artifacts/data_split/train.csv]
[2025-05-15 12:53:30,501: INFO: data_loader: Dataframe loaded successfully with 8712 records.]
[2025-05-15 12:53:30,502: INFO: data_loader: Loading dataframe from file: artifacts/data_split/val.csv]
[2025-05-15 12:53:30,511: INFO: data_loader: Dataframe loaded successfully with 1121 records.]
[2025-05-15 12:53:30,513: INFO: data_loader: Loading dataframe from file: artifacts/data_split/test.csv]
[2025-05-15 12:53:30,527: INFO: data_loader: Dataframe loaded successfully with 2613 records.]
[2025-05-15 12:53:31,735: INFO: data_loader: Valid file paths count: 8712]
[2025-05-15 12:53:31,851: INFO: data_loader: Valid file paths count: 1121]
[2

Initialized MLflow to track repo "DikshantPatel2210/KidneyDiseaseClassification-DLProject"

[2025-05-15 12:53:45,343: INFO: helpers: Initialized MLflow to track repo "DikshantPatel2210/KidneyDiseaseClassification-DLProject"]


Repository DikshantPatel2210/KidneyDiseaseClassification-DLProject initialized!

[2025-05-15 12:53:45,345: INFO: helpers: Repository DikshantPatel2210/KidneyDiseaseClassification-DLProject initialized!]
[2025-05-15 12:53:46,354: INFO: 3666370944: Logged parameters and metrics to MLflow.]


2025/05/15 12:53:46 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\DIKSHA~1\AppData\Local\Temp\tmp2l4oq7a5\model\data\model\assets
[2025-05-15 12:53:48,608: INFO: builder_impl: Assets written to: C:\Users\DIKSHA~1\AppData\Local\Temp\tmp2l4oq7a5\model\data\model\assets]


Registered model 'OptunaBestModel' already exists. Creating a new version of this model...
2025/05/15 12:54:07 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: OptunaBestModel, version 6


[2025-05-15 12:54:07,294: INFO: 3666370944: Model registered to MLflow Registry.]


Created version '6' of model 'OptunaBestModel'.


In [7]:
optuna_best_model = tf.keras.models.load_model("artifacts/optuna_best_model/best_model.keras")

In [12]:
optuna_best_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 112, 112, 32)      320       
                                                                 
 batch_normalization_1 (Batc  (None, 112, 112, 32)     128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 64)        18496     
                                                                 
 batch_normalization_2 (Batc  (None, 28, 28, 64)       256       
 hNormalization)                                                 
                                                      

In [32]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model  # in case you need to load the model
from pathlib import Path

# Set the directory containing images
img_dir = Path("artifacts/Random kidney CT scan images")
image_paths = list(img_dir.glob("*.jpg")) + list(img_dir.glob("*.png"))

# Load the model if not already loaded
# final_model = load_model("path_to_your_trained_model.h5")

# Parameters
target_size = (224, 224)
predictions = []

for img_path in image_paths:
    try:
        img = image.load_img(img_path, target_size=target_size, color_mode="grayscale")
        img_array = image.img_to_array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

        prediction = optuna_best_model.predict(img_array)
        pred_index = np.argmax(prediction)

        predictions.append({
            "filename": img_path.name,
            "prediction_index": int(pred_index),
            "probabilities": prediction.tolist()[0]  # Convert to list for readability/logging
        })

        print(f"{img_path.name} → Class {pred_index}, Probabilities: {prediction.tolist()[0]}")

    except Exception as e:
        print(f"Failed to process {img_path.name}: {e}")

1168.jpg → Class 3, Probabilities: [0.0006757155060768127, 0.0028174221515655518, 0.001478305202908814, 0.995028555393219]
istockphoto-1009903486-612x612.jpg → Class 0, Probabilities: [0.5347994565963745, 0.0003213557065464556, 0.36142751574516296, 0.10345160216093063]
istockphoto-1145702331-612x612.jpg → Class 3, Probabilities: [0.1841907799243927, 3.6089479635847965e-06, 0.05453867465257645, 0.7612670063972473]
istockphoto-1173159873-612x612.jpg → Class 2, Probabilities: [0.024105843156576157, 3.087478762608953e-05, 0.9756999015808105, 0.00016339738795068115]
istockphoto-1173165072-612x612.jpg → Class 0, Probabilities: [0.9829535484313965, 0.00016752778901718557, 0.011347481980919838, 0.005531479138880968]
istockphoto-1306894722-612x612.jpg → Class 3, Probabilities: [0.0005721120396628976, 0.022705284878611565, 0.0003431475779507309, 0.9763794541358948]
istockphoto-1716113526-612x612.jpg → Class 2, Probabilities: [0.10818924754858017, 0.009340640157461166, 0.8812748193740845, 0.00119

In [16]:
from tensorflow.keras.preprocessing import image

# Define the image path
img_path = "artifacts/data_ingestion/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone\Cyst\Cyst- (1029).jpg"

# Load and preprocess the image
img = image.load_img(img_path, target_size=(224, 224), color_mode="grayscale")  # match your training size
img_array = image.img_to_array(img)
img_array = img_array / 255.0  # normalize if you used rescale=1./255 during training
img_array = np.expand_dims(img_array, axis=0)  # add batch dimension

# Predict
prediction = optuna_best_model.predict(img_array)
pred_index = np.argmax(prediction)

In [17]:
pred_index

1